In [ ]:
#dataset merge
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# Step 1: Load and preprocess your dataset
# data = pd.read_csv('')  # Update with your dataset file path
# import dataset
pd.set_option('display.max_colwidth', 100)
data =pd.read_csv("D:\pythonana\dsadd.csv")
# Extract the text and label columns


In [ ]:
from bs4 import BeautifulSoup
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

# Download the stop words corpus
# nltk.download('stopwords')

stops = set(stopwords.words('english')) #english stopwords

stemmer = SnowballStemmer('english') #SnowballStemmer

def review_to_words(raw_review):
    # 1. Delete HTML 
    try:
        review_text = BeautifulSoup(raw_review, 'html.parser').get_text()
        # 2. Make a space
        letters_only = re.sub('[^a-zA-Z]', ' ', review_text)
        # 3. lower letters
        words = letters_only.lower().split()
        # 5. Stopwords 
        meaningful_words = [w for w in words if not w in stops]
        # 6. Stemming
        stemming_words = [stemmer.stem(w) for w in meaningful_words]
        # 7. space join words
        return( ' '.join(stemming_words))
    except:
        return "i"

In [ ]:
data.tail(15)

In [ ]:
data['Review'] = data['Review'].apply(review_to_words)

In [ ]:
# make proper modifications to the dataset
def giverate(i):
#     print(i)
#     if isdigit(i):
#         i=int(i)
    try:
        i=int(i)
    except ValueError:
        flag = False
    if i == 5:
        return 'positive'
    elif i==4:
        return 'semi-positive'
    elif i==3:
        return 'neutral'
    elif i==2:
        return 'semi-negative'
    else:
        return 'negative'

    
data['Sentiment'] = data['Rate'].apply(lambda x : giverate((x)))

In [ ]:
data.tail(15)

In [ ]:
texts = data['Review'].tolist()

labels = data['Sentiment'].tolist()

# Step 2: Encode the labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Step 3: Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
vocab_size = len(tokenizer.word_index) + 1
sequences = tokenizer.texts_to_sequences(texts)

# Step 4: Pad sequences to have equal length
max_len = max(len(seq) for seq in sequences)
X = pad_sequences(sequences, maxlen=max_len)

# Step 5: Split your dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, encoded_labels, test_size=0.2, random_state=42)

# Step 6: Build the CNN model
embedding_dim = 100
filters = 64
kernel_size = 5

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(Conv1D(filters, kernel_size, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Step 7: Train the model
batch_size = 32
epochs = 10
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=batch_size, epochs=epochs)

# Step 8: Evaluate the model
y_pred = np.argmax(model.predict(X_test), axis=-1)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Step 9: Decode the predicted labels
decoded_labels = label_encoder.inverse_transform(y_pred)


In [ ]:
# Preprocess the incoming data
incoming_text = "deserves more like"
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

# # Initialize the stemmer
# stemmer = PorterStemmer()

# # Example sentence
# # sentence = "The quick brown foxes jumped over the lazy dogs"

# # Tokenize the sentence
# tokens = word_tokenize(incoming_text)

# # Apply stemming to each token
# incoming_text = " ".join([stemmer.stem(token) for token in tokens])

# Print the stemmed words
print(incoming_text)
incoming_sequence = tokenizer.texts_to_sequences([incoming_text])
incoming_padded_sequence = pad_sequences(incoming_sequence, maxlen=max_len)

# Predict the sentiment label for the incoming data
predicted_label = np.argmax(model.predict(incoming_padded_sequence), axis=-1)

# Decode the predicted label
decoded_label = label_encoder.inverse_transform(predicted_label)[0]

# print("Incoming Text:", incoming_text)
print("Predicted Label:", decoded_label)

In [ ]:
# Step 10: Save the trained model
import tensorflow as tf
tf.autograph.set_verbosity(0)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf

# Set Autograph verbosity to suppress the warning
tf.autograph.set_verbosity(0)

# Rest of your code

# Save the model
# model.save('D:\pythonana')



In [ ]:
# Load the saved model
# Save the model
model.save("model.h5")


In [ ]:
# Load the model
loaded_model = tf.keras.models.load_model("model.h5")


In [ ]:
# Preprocess the new data
new_texts = ['good','bad','as expected']
new_sequences = tokenizer.texts_to_sequences(new_texts)
new_padded_sequences = pad_sequences(new_sequences, maxlen=max_len)

# Perform predictions using the loaded model
predictions = loaded_model.predict(new_padded_sequences)

# Decode the predicted labels
decoded_labels = label_encoder.inverse_transform(np.argmax(predictions, axis=-1))

# Print the predictions
for text, label in zip(new_texts, decoded_labels):
    print(f'Text: {text}\nPredicted Label: {label}\n')


In [ ]:
# save tokenizer
import pickle

# Save the tokenizer
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)


In [ ]:
#save label_encoder = LabelEncoder()
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)


In [ ]:
# predict the model
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

# Load the saved model
loaded_model = tf.keras.models.load_model("D:\pythonana\model.h5")

# Load the tokenizer
with open(r"D:\pythonana\tokenizer.pkl", 'rb') as f:
    tokenizer = pickle.load(f)

# Load the label_encoder
with open(r'D:\pythonana\label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)




# Define your new text data
new_texts = ['New text 1', 'New text 2', 'New text 3']

# Tokenize the new text data
sequences = tokenizer.texts_to_sequences(new_texts)
max_len = loaded_model.input_shape[1]
padded_sequences = pad_sequences(sequences, maxlen=max_len)

# Make predictions on the new data
predictions = loaded_model.predict(padded_sequences)

# Decode the predictions
decoded_predictions = label_encoder.inverse_transform(np.argmax(predictions, axis=-1))



# Print the predictions
for text, prediction in zip(new_texts, decoded_predictions):
    print(f'Text: {text}')
    print(f'Prediction: {prediction}')
    print()

In [ ]:
# deploy


from flask import Flask, request
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import pickle

app = Flask(__name__)

# Load the model and tokenizer
# Load the saved model
loaded_model = tf.keras.models.load_model("D:\pythonana\model.h5")

# Load the tokenizer
with open(r"D:\pythonana\tokenizer.pkl", 'rb') as f:
    tokenizer = pickle.load(f)

# Load the label_encoder
with open(r'D:\pythonana\label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)


@app.route('/', methods=['GET'])
def hello_world():
    incoming_text = "worst one i ever seen"
    # Preprocess the incoming text
    incoming_sequence = tokenizer.texts_to_sequences([incoming_text])
    incoming_padded_sequence = pad_sequences(incoming_sequence, maxlen=65)

    # Predict the sentiment label for the incoming data
    predicted_label = np.argmax(loaded_model.predict(incoming_padded_sequence), axis=-1)

    # Decode the predicted label
    decoded_label = label_encoder.inverse_transform(predicted_label)[0]

    # Return the prediction as the response
    return 'Prediction: ' + decoded_label

if __name__ == '__main__':
    app.run()


In [ ]:
# from flask import Flask, request, jsonify
# import tensorflow as tf
# import numpy as np
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.preprocessing.text import Tokenizer
# import pickle

# app = Flask(__name__)

# # Load the model and tokenizer
# # Load the saved model
# loaded_model = tf.keras.models.load_model("D:\pythonana\model.h5")

# # Load the tokenizer
# with open(r"D:\pythonana\tokenizer.pkl", 'rb') as f:
#     tokenizer = pickle.load(f)

# # Load the label_encoder
# with open(r'D:\pythonana\label_encoder.pkl', 'rb') as f:
#     label_encoder = pickle.load(f)


# @app.route('/', methods=['POST'])
# def predict_sentiment():
#     if request.method == 'POST':
#         data = request.json  # assuming the request body contains JSON data
#         incoming_text = data.get('text', '')

#         # Preprocess the incoming text
#         incoming_sequence = tokenizer.texts_to_sequences([incoming_text])
#         incoming_padded_sequence = pad_sequences(incoming_sequence, maxlen=1000)

#         # Predict the sentiment label for the incoming data
#         predicted_label = np.argmax(loaded_model.predict(incoming_padded_sequence), axis=-1)

#         # Decode the predicted label
#         decoded_label = label_encoder.inverse_transform(predicted_label)[0]

#         # Return the prediction as the response
#         return jsonify({'prediction': decoded_label})

#     return 'Method not allowed', 405


# if __name__ == '__main__':
#     app.run()


In [ ]:
#  prediction if the model saves tokenizer and model

# #predictions

# import tensorflow as tf
# from tensorflow.keras.preprocessing.sequence import pad_sequences

# # Load the saved model
# loaded_model = tf.keras.models.load_model("D:\pythonana\model.h5")

# # Define your new text data
# new_texts = ['New text 1', 'New text 2', 'New text 3']

# # Tokenize the new text data
# tokenizer = loaded_model.tokenizer
# sequences = tokenizer.texts_to_sequences(new_texts)
# max_len = loaded_model.max_len
# padded_sequences = pad_sequences(sequences, maxlen=max_len)

# # Make predictions on the new data
# predictions = loaded_model.predict(padded_sequences)

# # Decode the predictions
# decoded_predictions = loaded_model.label_encoder.inverse_transform(predictions)

# # Print the predictions
# for text, prediction in zip(new_texts, decoded_predictions):
#     print(f'Text: {text}')
#     print(f'Prediction: {prediction}')
#     print()

In [ ]:
# model if not saved in .h5 format


# from flask import Flask, request
# import tensorflow as tf
# import numpy as np
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.preprocessing.text import Tokenizer

# app = Flask(__name__)

# # Load the model and tokenizer
# loaded_model = tf.keras.models.load_model("path/to/your/model")
# tokenizer = Tokenizer()
# tokenizer.load("path/to/your/tokenizer")

# @app.route('/', methods=['GET'])
# def hello_world():
#     incoming_text = 'i love this'

#     # Preprocess the incoming text
#     incoming_sequence = tokenizer.texts_to_sequences([incoming_text])
#     incoming_padded_sequence = pad_sequences(incoming_sequence, maxlen=1000)

#     # Predict the sentiment label for the incoming data
#     predicted_label = np.argmax(loaded_model.predict(incoming_padded_sequence), axis=-1)

#     # Decode the predicted label
#     decoded_label = label_encoder.inverse_transform(predicted_label)[0]

#     # Return the prediction as the response
#     return 'Prediction: ' + decoded_label

# if __name__ == '__main__':
#     app.run()
